In [1]:
import folium
import DownloadData as DD
import DataSorting as DS
import json
import geojson
from urllib.request import urlopen
from geojson_utils import centroid

In [51]:
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/cantons.geojson') as f:
    stations = geojson.load(f)

In [ ]:
DD.get_ten_Mins_Data()
DD.get_dailyData()
st_prices = DS.DictCantons("PrixCarburants_instantane.xml")
chaine = DD.get_chaine()
#Dictionnaires prix (Instantané)
st_prices_B = DS.DictCantons("PrixCarburants_quotidien_" + chaine + ".xml")

In [39]:
variations = {key: st_prices[key] - st_prices_B.get(key, 0) for key in st_prices.keys()}

In [ ]:
t = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

for i in range (0, len(stations["features"])):
    if stations["features"][i].geometry.type == 'Polygon':
        box_str = str(stations["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        nom_v = stations["features"][i]["properties"]["nom"]
        if nom_v in variations:
            if variations[nom_v] < 0:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_v}</br>{round(variations[nom_v], 3)}',
                                  icon = folium.features.CustomIcon('down.png',icon_size=(15,15))).add_to(t)
            elif variations[nom_v] == 0:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_v}</br>{round(variations[nom_v], 3)}',
                                  icon = folium.features.CustomIcon('equal.png',icon_size=(15,15))).add_to(t)
            else:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_v}</br>+{round(variations[nom_v], 3)}',
                                  icon = folium.features.CustomIcon('up.png',icon_size=(15,15))).add_to(t)

t